<a href="https://colab.research.google.com/github/jinsyabeycg/BITS-AIML-Mtech-V2/blob/master/Group_136_NLP_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Group ID:
### Group Members Name with Student ID:
1. Nimishkumar Patel
2. Jinsy Abey
3. Sunil Pandey
4. Sufiya
5. Ashwani
